# EDA Binary Classification with a Bank Dataset

### This EDA refers to Kaggle playground competition: "Binary Classification with a Bank Dataset" 
- Competition link: https://www.kaggle.com/competitions/playground-series-s5e8
- Main goal of this competition is to predict whether a client will subscribe to a bank term deposit.
- Submissions are evaluated using ROC AUC between the predicted value and the observed target.
- The dataset for this competition (both train and test) was generated from a deep learning model trained on the Bank Marketing Dataset dataset. Feature distributions are close to, but not exactly the same, as the original.
- Start Date - August 1, 2025
- Final Submission Deadline - August 31, 2025

### Labels detail from original data set

- age: Age of the client (numeric)
- job: Type of job (categorical: "admin.", "blue-collar", "entrepreneur", etc.)
- marital: Marital status (categorical: "married", "single", "divorced")
- education: Level of education (categorical: "primary", "secondary", "tertiary", "unknown")
- default: Has credit in default? (categorical: "yes", "no")
- balance: Average yearly balance in euros (numeric)
- housing: Has a housing loan? (categorical: "yes", "no")
- loan: Has a personal loan? (categorical: "yes", "no")
- contact: Type of communication contact (categorical: "unknown", "telephone", "cellular")
- day: Last contact day of the month (numeric, 1-31)
- month: Last contact month of the year (categorical: "jan", "feb", "mar", …, "dec")
- duration: Last contact duration in seconds (numeric)
- campaign: Number of contacts performed during this campaign (numeric)
- pdays: Number of days since the client was last contacted from a previous campaign (numeric; -1 means the client was not previously contacted)
- previous: Number of contacts performed before this campaign (numeric)
- poutcome: Outcome of the previous marketing campaign (categorical: "unknown", "other", "failure", "success")
- y: The target variable, whether the client subscribed to a term deposit

In [ ]:
import numpy as np 
import pandas as pd 
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

warnings.filterwarnings("ignore",  category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning,)
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
sub_df = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

## 1. Basic check of data

In [ ]:
display(train_df.head(4))
display(test_df.head(4))

In [ ]:
display(train_df.info())
display(test_df.info())

In [ ]:
display("Summary for TRAIN data:",train_df.describe())
display("Summary for TEST data:",test_df.describe())

In [ ]:
display("TRAIN data", train_df.nunique())
print('\n')
display('TEST data',test_df.nunique())

### Key Observations:
- We have 7 numeric columns and 9 with categories
- month column can be mapped to numerical column if needed
- One coulmn with id in both train/test sets and additional column with labels in train set
- number of unique values different for columns age, balance, duration, campaign, pdays, previous
- min values different for train - test set for column duration
- max values different balance, campaign, previous
- pdays have two separate information - first number of days and second == -1 means the client was not previously contacted. Can be separated into two columns to separate category from numeric

### BONUS: Dimensions reduction with t-SNE

In [ ]:
X = train_df.drop(columns=['id', 'y'])
y = train_df['y']

# Separation of numeric and categorical columns
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Pipeline: przetwarzanie cech
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols)
])

X_processed = preprocessor.fit_transform(X)

# Dimensions reduction with t-SNE
# t-SNE is very slow for large datasets. We will check only 20k random samples
SAMPLE_SIZE = 20000

idx = np.random.choice(len(X_processed), SAMPLE_SIZE, replace=False)
X_sample = X_processed[idx]
y_sample = y.iloc[idx]

tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(X_sample)


# Creating plot
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=X_tsne[:, 0],
    y=X_tsne[:, 1],
    hue=y_sample,
#    palette='coolwarm',
    s=5,
    alpha=0.6,
    linewidth=0
)
plt.title("t-SNE 2D wizualizacja danych (kolor = y)")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title='y')
plt.grid(True)
plt.tight_layout()
plt.show()

### 2nd BONUS - Separation of pdays and additional column wuth months as numeric

In [ ]:
# we can create separate column with flag for -1 value
train_df['no_previous_contact'] = (train_df['pdays'] == -1).astype(int)
test_df['no_previous_contact'] = (test_df['pdays'] == -1).astype(int)

# We can create additional column with pdays only without -1 values
train_df['pdays_cleaned'] = train_df['pdays'].where(train_df['pdays'] != -1, np.nan) 
test_df['pdays_cleaned'] = test_df['pdays'].where(test_df['pdays'] != -1, np.nan) 

# We can create additional column with numeric months
train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
test_df['month_as_num'] = test_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})

## 2. Duplicates and missing values check

In [ ]:
print("Duplicates in TRAIN data:", train_df.duplicated().sum())
print("Duplicates in TEST data:", test_df.duplicated().sum())

In [ ]:
print("Missing values in TRAIN data:\n",train_df.isna().mean().apply(lambda x: f"{x:.2%}"))
print("\nMissing values  in TEST data:\n",test_df.isna().mean().apply(lambda x: f"{x:.2%}"))

### Key Observations:
- There is no duplicates in train/test data
- There are no missing data in train/test sets
- in case we decide to clean pdays we have missing 89,66% of data 


## 3. Train-Test drift check

### Numeric column drift

In [ ]:
for col in test_df.columns:
    if col != 'id' and test_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(train_df[col], label='test', fill=True)
        sns.kdeplot(test_df[col], label='train', fill=True)
        plt.title(f"Drift check for Column: {col}")
        plt.legend()
        plt.show()

In [ ]:
feature ='balance'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='duration'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='campaign'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='pdays'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.9)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='previous'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.999)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Categorical columns drift check

In [ ]:
def plot_category_drift(feature):
    pd.concat([
        train_df[feature].value_counts(normalize=True).rename("train"),
        test_df[feature].value_counts(normalize=True).rename("test")
    ], axis=1).plot(kind="bar", title=f"Category drift: {feature}")

columns = [ 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact','month', 'poutcome'  ]

for col in columns:
    plot_category_drift(col)

### Key Observations:
- visible drift for column 'previous'
- no clear drift between train and test data for rest of numeric columns
- no clear drift between train and test data for categorical columns

## 4. Correlation check for train and test data

In [ ]:
sns.heatmap(train_df[['age', 'balance','day', 'duration','campaign', 'pdays','previous', 'month_as_num', 'pdays_cleaned']].corr(),
            annot = True, cmap='coolwarm')

In [ ]:
sns.heatmap(test_df[['age', 'balance','day', 'duration','campaign', 'pdays','previous','month_as_num', 'pdays_cleaned']].corr(), 
            annot = True, cmap='coolwarm')

### Key Observations:
- Only pdays and previous columns show strong correlation
- possible weak negative correlation between new columns - month_as_num and pdays_cleaned

## 5. Check of 'y' in each column for train data

### Category columns

In [ ]:
sns.countplot(data=train_df, x='y')
round(train_df['y'].value_counts(normalize=True)*100,2)

In [ ]:
columns = [ 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',  'poutcome', 
            'month_as_num', 'no_previous_contact']
for col in columns:
    train_df.groupby([col,'y']).size().unstack().plot(kind='bar', stacked=True, title=col)
    plt.show()
    print('Percentage summary:')
    display((pd.crosstab(train_df[col], train_df["y"], normalize='index') * 100).round(1))
    print('Quantitative summary:')
    display((pd.crosstab(train_df[col], train_df["y"])))

### Key Observations:
- We have 2 gropus in label column with split 88% 12% - strong unbalance of classes
- Each categorical column show caategories with different ratio of 0 / 1 labels and have potential to be used in classification


### Numerical columns

In [ ]:
columns = [ 'age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous',  'pdays_cleaned', 'month_as_num']

for feature in columns:
    plt.figure(figsize=(8, 4))
    sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, label='y = 0', fill=True, alpha=0.4)
    sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, label='y = 1', fill=True, alpha=0.4)
    plt.title(f'KDE for {feature}')
    #plt.xscale('log')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Key Observations:
- Most of columns show differences in density for labels 0 an 1 and have good potential to be used for classification - age, day, duration, pdays_cleaned, month_as_num
- month column seems to not show cyclical behaviour so it can be probably used as categorical column
- day seems to have no cyclical behaviour so it can be also consider as categorical however 31 categories can be quite large number - to be checked
- new column pdays_cleaned show differences between 0 and 1 but it consider about 11% of data, rest of them is -1
- some plots are not so godd visible so they can be analysed in more details like previous

### Additional plots for more details

In [ ]:
feature ='balance'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='duration'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.995)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='campaign'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.97)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom  for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='previous'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.995)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Key Observations:
- Balance column show small differences in density - can be verified with feature importance
- duration column show quite good differences in density - it is good potential for classification
- campaign column seems to have no bigger differences -  can be verified with feature importance
- for column previous KDE seems to show much different behaviour and seems to have a very good potential for classification

## 6. Outliners for numerical columns

In [ ]:
columns = [ 'age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous',  'pdays_cleaned', 'month_as_num']

for col in columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x='y', y=col, data=train_df)
    plt.title(f'Boxplot of {col} by y label column')
    plt.show()

### Key Observations:
- It seems ...........................................

## 7. Ideas for feature engineering

### column job - grouping by income/Social status/ stability of employment


### poutcome: Bardzo ważna zmienna – warto zostawić jako osobną kolumnę, ale też stworzyć zmienną binarną typu: prev_success = poutcome == 'success'.


### b) Transformacje logarytmiczne / winsoryzacja
balance, duration, pdays, previous mogą mieć rozkład z ogonami – warto sprawdzić wykresy. Spróbuj:

log(1 + balance), log(1 + duration), log(1 + previous)

Winsoryzacja (obcięcie ekstremalnych wartości)


### Interactions between columns

poutcome * previous 

age * duration

job + education

contact * month

### Grouping in columns

Grupowanie wieku
age_group = 'young' (<=30) / 'middle' (31–60) / 'senior' (>60) – może mieć różny wpływ na decyzje kredytowe.

c) Zmienna sezonowa
is_summer = month in ['jun', 'jul', 'aug'] – kampanie w wakacje mogą mieć inną skuteczność.

d) Długość kontaktu / skuteczność kontaktu
long_contact = duration > X (np. 120 sekund)

campaign_efficiency = previous / (1 + campaign) – czy wiele kontaktów wcześniej przynosiło efekt?

Zobowiązania finansowe
loan_sum = (housing == 'yes') + (loan == 'yes') – liczba aktywnych pożyczek

is_deep_debt = (balance < 0) & (loan_sum > 1) – mocno zadłużony

Poziom edukacji + zawód
edu_job = education + "_" + job – np. "tertiary_admin"

In [ ]:
👫 Małżeństwo vs. wiek
is_young_single = (age < 30) & (marital == 'single')

is_old_married = (age > 60) & (marital == 'married')

📞 3. Komunikacja i kanał kontaktu
🔔 Efektywność kanału kontaktu
contact_success_rate = campaign / (duration + 1) – ile kontaktów na minutę

🛠️ Rodzaj komunikacji + sukces poprzedni
channel_prev_success = contact + "_" + poutcome

In [ ]:
sns.countplot(data=train_df,  x='housing', hue='y')

In [ ]:
sns.boxplot(data=train_df, x='education', y='pdays_cleaned')

In [ ]:
sns.boxplot(data=train_df, x='marital', y='age', hue='y')

In [ ]:
sns.pairplot(train_df[[ 'age','balance', 'pdays_cleaned','y']], hue='y') 

1. Numeryczne ↔ Numeryczne
Pairplot (sns.pairplot)
Kilka zmiennych naraz – przegląd zależności i gęstości
sns.pairplot(df, hue='label')  # opcjonalnie hue dla klasy binarnej

Scatterplot z hue / style / size
Super do odkrywania nieliniowych relacji
sns.scatterplot(data=df, x='age', y='balance', hue='loan', style='marital')

Heatmap korelacji (sns.heatmap)
Pokazuje mocne lub słabe powiązania liniowe
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')

2. Kategoryczne ↔ Numeryczne
Boxplot (sns.boxplot)
Widać mediany, rozrzut i outliery
sns.boxplot(data=df, x='education', y='balance')

Violinplot (sns.violinplot)
To samo co boxplot, ale z KDE – pokazuje lepiej gęstości
sns.violinplot(data=df, x='job', y='age', hue='y', split=True)

Swarmplot (sns.swarmplot)
Każdy punkt osobno – dobry na małych zbiorach
sns.swarmplot(data=df, x='marital', y='balance')

Barplot (sns.barplot)
Średnia wartość cechy numerycznej dla kategorii (np. średni balance dla job)
sns.barplot(data=df, x='job', y='balance')

 3. Kategoryczne ↔ Kategoryczne
Heatmap cross-tab (pd.crosstab + sns.heatmap)
Ile wystąpień danego połączenia
ct = pd.crosstab(df['job'], df['marital'])
sns.heatmap(ct, annot=True, fmt='d', cmap='Blues')

Countplot (sns.countplot)
Liczba obserwacji w każdej kategorii + hue np. y
sns.countplot(data=df, x='education', hue='y')

4. Numeryczne ↔ Target binarny
KDE Plot (sns.kdeplot)
Rozkład cechy w dwóch klasach
sns.kdeplot(data=df[df['y'] == 0], x='age', label='No Loan')
sns.kdeplot(data=df[df['y'] == 1], x='age', label='Loan Taken')

Histogram + hue
sns.histplot(data=df, x='balance', hue='y', bins=30, kde=True, stat='density')
🔹 5. Dodatkowe / Interaktywne
✅ FacetGrid
Podzielone wykresy np. według marital i education
g = sns.FacetGrid(df, col="marital", row="education", hue="y")
g.map(sns.kdeplot, "age", fill=True)

jointplot (sns.jointplot)
Scatter + marginesowe rozkłady
sns.jointplot(data=df, x='balance', y='duration', hue='y', kind='kde')



## 8. XXXXXXXXXXXXXXXXXXXXXXXXXXXXX

### Key Observations:
- Stage_fear and Drained_after_socializing columns seems to be strong indicator of classification
- In each column there is small representation of opposite group prefference (Extrovert preffers No & No, but there is small group of introverts with same prefferences)
- It is very rare situation to have Yes - No and No - Yes answers. It is very specific minor group
- It may be resonable to impute data Yes when other column is Yes and oposite for No, for such rare cases
